# モジュールをインポート

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# モデルを定義

In [3]:
from tensorflow.keras.models import load_model

In [4]:
results_dir = "./results/keras_2019_12_23_12_11"

In [5]:
model = load_model(results_dir + "/model/model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# モデルの確認

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 10, 11, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 10, 11, 128)   1280      
_________________________________________________________________
reshape (Reshape)            (None, 10, 14080)         0         
_________________________________________________________________
dense (Dense)                (None, 10, 1024)          14418944  
_________________________________________________________________
dropout (Dropout)            (None, 10, 1024)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 1024)              629452

In [7]:
# opt = Adam(lr=learning_rate)
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## 学習済みモデルの重みを読み込み

In [8]:
# checkpoint_path = "./results/keras_2019_11_21/keras_model/cp.ckpt"
# model.load_weights(checkpoint_path)

# Load data stream

In [9]:
# from pylsl import StreamInlet, resolve_byprop, local_clock
# from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK
# from datetime import datetime
# LSL_EEG_CHUNK = 10
# def record_stream():
#     """
#     :return: a generator to fetch EEG data from existed stream.
#     """
#     streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
#     if len(streams) == 0:
#         raise IOError("Can't find EEG stream.")

#     inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

#     while True:
#         chunk, timestamp = inlet.pull_sample(timeout=1.0)
#         if timestamp:
#             timestamp = datetime.fromtimestamp(timestamp).strftime("%A, %B %d, %Y %I:%M:%S")
#             print(timestamp, end="\r", flush=True)
# record_stream()

In [10]:
from pylsl import StreamInlet, resolve_byprop
from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK

LSL_EEG_CHUNK = 10
def record_stream():
    """
    :return: a generator to fetch EEG data from existed stream.
    """
    streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
    if len(streams) == 0:
        raise IOError("Can't find EEG stream.")

    inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

    while True:
        yield inlet.pull_chunk(timeout=1.0, max_samples=LSL_EEG_CHUNK)


In [11]:
samples, timestamps = next(record_stream())
print(samples)

[[-1000.0, 538.57421875, -730.46875, -1000.0, -868.65234375], [-525.87890625, -978.02734375, 581.54296875, -802.24609375, 863.28125], [-944.3359375, -404.78515625, -993.65234375, 791.9921875, -687.98828125], [292.48046875, -1000.0, -267.578125, 838.37890625, -604.00390625], [-1000.0, -26.85546875, -889.16015625, -711.42578125, 844.7265625], [-1000.0, -794.921875, -925.78125, -1000.0, -809.08203125], [409.1796875, 117.1875, -397.94921875, 74.70703125, -79.1015625], [537.109375, -1000.0, -479.98046875, -786.62109375, -837.890625], [-954.1015625, -252.9296875, 992.67578125, -14.6484375, 593.26171875], [-1000.0, -798.33984375, -800.29296875, -1000.0, -751.953125]]


In [12]:
print(timestamps)

[1577198621.586767, 1577198621.5906732, 1577198621.5945795, 1577198621.5984857, 1577198621.602392, 1577198621.6062982, 1577198621.6102045, 1577198621.6141107, 1577198621.618017, 1577198621.6219232]


In [13]:
# print("['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX']")
# print(np.array(next(record_stream())[0])[:, 0:4])

# Preprocessing

In [14]:
def data_1Dto2D(data, Y=10, X=11):
    data_2D = np.zeros([Y, X])
    data_2D[1, 3] = data[1]
    data_2D[1, 7] = data[2]
    data_2D[5, 1] = data[0]
    data_2D[5, 9] = data[3]
# 	data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0,        0,        0,        0, 	   0,  	     0, 	   0, 	 	 0) 
# 	data_2D[1] = (	  	 0, 	   0,  	   	 0, data[1],        0,        0,        0, data[2], 	   	 0,   	   0, 	 	 0) 
# 	data_2D[2] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 	 	 0) 
# 	data_2D[3] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[4] = (       0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0) 
# 	data_2D[5] = (	  	 0, data[0],        0,        0,        0,        0,        0,        0,        0, data[3], 		 0) 
# 	data_2D[6] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[7] = (	  	 0, 	   0, 	 	 0,        0,        0,        0,        0,        0, 	   	 0, 	   0, 		 0) 
# 	data_2D[8] = (	  	 0, 	   0, 	 	 0, 	   0,        0,        0,        0, 	   0, 	   	 0, 	   0, 		 0) 
# 	data_2D[9] = (	  	 0, 	   0, 	 	 0, 	   0, 	     0,        0, 		 0, 	   0, 	   	 0, 	   0, 		 0) 
    return data_2D

In [15]:
true_labels = np.array(['eye_close', 'image_open&close_both_feet',
       'image_open&close_both_fists', 'image_open&close_left_fist',
       'image_open&close_right_fist'])

In [16]:
def feature_normalize(data):
	mean = data[data.nonzero()].mean()
	sigma = data[data.nonzero()].std()
	data_normalized = data
	data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
	return data_normalized

In [40]:
from IPython.display import clear_output
# print("23:51:39:746923  [[False False  True False False]]")
# print("   Timestamp##################Results#############")
print("Timestamp          Results")

for stream, timestamps in record_stream():
    buffer_3d = []
    buffer_2d = np.zeros((10, 11))
    num = 0
    for data in stream:
        data = np.array(data)
        data = feature_normalize(data)
        buffer_2d[1, 3] = data[1]
        buffer_2d[1, 7] = data[2]
        buffer_2d[5, 1] = data[0]
        buffer_2d[5, 9] = data[3]
        buffer_3d.append(buffer_2d)
    buffer_3d = np.array(buffer_3d)
    X_test = buffer_3d.reshape((1, 10, 10, 11, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
#     print(y_pred.round(3), end="\r", flush=True)
    y_pred = (y_pred > 0.8)
    print(datetime.fromtimestamp(timestamps[-1]).strftime("%H:%M:%S:%f")
          + "  " + str(y_pred), end="\r", flush=True)
#     print(y_pred, end="\n")
#     print(flush=True)
#     y_pred_bool = np.argmax(y_pred, axis=1)
#     print(true_labels[int(y_pred_bool)], end="\r", flush=True)
#     print(true_labels[int(y_pred_bool)])

Timestamp          Results


KeyboardInterrupt: 

In [ ]:
for stream in record_stream():
    buffer_2d = np.zeros((10, 10, 11))
#     print(stream[0])
    for i in next(record_stream())[0]:
        num = 0
        buffer_2d[num, 1, 3] = i[1]
        buffer_2d[num, 1, 7] = i[2]
        buffer_2d[num, 5, 1] = i[0]
        buffer_2d[num, 5, 9] = i[3]
    X_test = buffer_2d.reshape((1, 10, 10, 11, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(true_labels[int(y_pred_bool)])

# テストデータを分類

In [ ]:
y_pred = model.predict(X_test, batch_size=1, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

In [ ]:
print("サンプリング率:", 160)
print("1秒に処理できる信号数：", 990*10-6)

In [ ]:
y_pred_bool = one_hot_encoder(y_pred_bool)
y_pred_bool

# 精度と正解率の曲線

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.preprocessing import label_binarize

precision = dict()
recall = dict()
for i in range(5):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_pred_bool[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()

# 分類結果

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_bool))

In [ ]:
print("検証正解率：", history["val_acc"][-1])

In [ ]:
from sklearn.metrics import accuracy_score
print("テスト正解率：", accuracy_score(y_test, y_pred_bool))